In [49]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from sentence_transformers import SentenceTransformer


import sys
sys.path.append("/home/rtue/playground/jupyter-notebook/workspace/codes")
sys.path.append("/home/rtue/playground/jupyter-notebook/")
sys.path.append("/home/rtue/playground/python_projects/tool_keaml/src/utils")
import util, evaluation_metrics
from training_data.data_prepare import DataPrepare
import prepare_input_for_YouTubeDNN as prepare_input_for_dnn

import ast
import pandas as pd
from termcolor import colored
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer

In [50]:
path_dw_id_kw='/home/rtue/playground/python_projects/resources/df_np_id_kw.csv'
path_dw_id_tit_abst_claim ='/home/rtue/playground/python_projects/resources/df_np_id_tit_abst_claim_detden.csv'

In [51]:
f_name_test_dict='/home/rtue/playground/jupyter-notebook/obj/77_kws/train_test_sets/test_set_dict'
#/77_kws/train_test_sets/test_set_dict_30_de_20220222-144213'
#test_set_dict_10_de_20220222-134947'
#test_set_dict_above_70_de_20220221-175613'
# test_set_dict_30_de_20220222-144213

test_set_dict=util.load_obj(f_name_test_dict)

In [8]:
n_less_occ_th=100

In [9]:
data=DataPrepare()

data.set_der_abst()
data.set_full_text()

DataPrepare object has the following properties:
 df_full_text, dict_full_text, df_der_text, dict_der_text


In [10]:
rows=[]
for (k,v) in test_set_dict.items():
    rows.append([k,data.dict_der_text[k],list(v)])
    

df = pd.DataFrame(rows, columns=["PPID","DWPI_ABSTRACT", "KEYWORD"]) 
df.to_csv('df_test.csv',sep='\t', index=False)

In [11]:
df_tmp=pd.read_csv('df_test.csv',sep='\t', error_bad_lines=False)

In [12]:
df_tmp_ft=pd.read_csv('df_test_fulltext.csv',sep='\t', error_bad_lines=False)

In [13]:
df_tmp_ft[df_tmp_ft['PPID']=='US20120276100A1']['KEYWORD'].iloc[0]

"['pancreatic cancer', 'breast cancer', 'alzheimer disease', 'prostate cancer', 'ovarian cancer', 'systemic lupus erythematosus', 'crohn disease', 'therapeutic agent', 'bladder cancer', 'multiple sclerosis', 'lung cancer', 'autoimmune disease', 'rheumatoid arthritis', 'liver cancer', 'ulcerative colitis', 'sjogren syndrome', 'cancer cell', 'colon cancer', 'neurodegenerative disease', 'myasthenia gravis']"

In [14]:
df_tmp[df_tmp['PPID']=='US20120276100A1']['DWPI_ABSTRACT'].iloc[0]

"A dock and lock (DNL) construct comprising an ribonuclease (RNase) attached to a dimerization and docking domain (DDD) moiety from human protein kinase A (PKA) RI- alpha , RI- beta , RII- alpha or RII- beta , and an antibody or antigen binding antibody fragment attached to an anchoring domain (AD) moiety from an A-kinase anchoring proteins (AKAP), where two copies of the DDD moiety form a dimer and bind to the AD moiety to form the DNL construct, is new.The DNL construct is useful for treating a disease by administering DNL construct in a subject, where the disease is chosen from cancer, immune dysfunction and autoimmune disease. The cancer is chosen from non-Hodgkin's lymphoma, B cell lymphoma, B cell leukemia, T cell lymphoma, T cell leukemia, acute lymphoid leukemia, chronic lymphoid leukemia, Burkitt's lymphoma, Hodgkin's lymphoma, hairy cell leukemia, acute myeloid leukemia, chronic myeloid leukemia, multiple myeloma, glioma, Waldenstrom's macroglobulinemia, carcinoma, melanoma, 

In [17]:
data.df_full_text.head()

,PPID,TIEN,ABEN,CLMEN,DETDEN
0,KR99044039A,Retrovirus packaging system,['The invention provides a novel retroviral pa...,"[""Recombinant Retrovirus in human cells for pr...",['The present invention refers to novel Retrov...
1,KR2002003917A,FUSION PROTEIN OF TAG PROTEIN AND HUMAN INTERL...,['PURPOSE: A fusion protein of tag protein and...,"['Thioredoxin (thioredoxin) region, to Proteol...",['Figure 1 shows a also C-terminal secretion i...
2,CN110088285A,SPATIOTEMPORAL REGULATORS,"['Provided herein, in some embodiments, are me...","["". Engineered genetic construct It comprises ...","['Related application', '@datdatdatas accordin..."
3,CN111378048A,Antibody-peptide bispecific immunotherapeutic ...,['The invention belongs to the technical field...,['.A fusion protein characterized in that the ...,"['Technical Field', 'The invention belongs to ..."
4,CN107532214A,METHODS FOR DIAGNOSIS OF SEPSIS,['Methods for diagnosis of sepsis are disclose...,['. A method for diagnosing a subject method o...,['With regard to the Federal-SPONSORED researc...


In [18]:
data_id_kw, data_id_tit_abs_clm, data_kw=util.read_dataset(path_dw_id_kw, path_dw_id_tit_abst_claim)

rows_with_NaN: Empty DataFrame
Columns: [PPID, KEYWORD]
Index: []
rows_with_NaN: Empty DataFrame
Columns: [PPID, TIEN, ABEN, CLMEN, DETDEN]
Index: []
data_kw is merged version and len(data_kw): 271137


In [19]:
data_id_tit_abs_clm.head()

,PPID,TIEN,ABEN,CLMEN,DETDEN
0,CN103627821A,"{'rt-lamp nucleic acid test strip', 'transmiss...","{'rt-lamp nucleic acid test strip', 'loopb pri...","{'loopb primer', 'porcine transmissible gastro...","{'lamp', 'agarose gel electrophoresis', 'optim..."
1,US20140121990A1,"{'genomic-enabled medicine', 'informatic infra...","{'generic computational technique', 'fully seq...","{'match', 'base protocol', 'pharmaceutical dru...","{'deployment', 'error', 'usc', 'pppmt', 'speci..."
2,WO2014064534A2,"{'immunogenic composition', 'injection needle'}","{'needle assembly', 'immune response', 'active...","{'polynucleotide encoding', 'polynucleotide', ...","{'percent', 'clearance', 'ross goose avihepadn..."
3,WO2014070975A1,"{'theil disease', 'flavivirus'}","{'theiler disease-associated virus', 'disease'...","{'host cell', 'seq ed nos', 'nucleotide', 'seq...","{'lysosensor', 'monobody', 'usa', 'serum plasm..."
4,US20140127259A1,{'cattle'},"{'heifer', 'cow', 'vaccinate cattle', 'against...","{'modify live virus', 'immunogenic composition...","{'tissue culture', 'hydroxyl group', 'sample v..."


In [20]:
dict_occ_kw=util.get_occurance_dict(data_kw,['KEYWORD'])

occurance dictionary len: 79450


In [21]:
list_ele_1=util.get_list_ele_len_less_than(dict_occ_kw)
list_ele_2=util.get_list_ele_occ_less_than(dict_occ_kw, n_less_occ_th)
list_ele_3 = util.get_list_pha_len_less(dict_occ_kw)
list_ele_4 = util.get_list_ele_occ_more_than(dict_occ_kw)
lst_all=set(list_ele_1 + list_ele_2+list_ele_3+list_ele_4)

In [22]:
data_kw=util.remove_rows_contain_kw_in_list(lst_all, data_kw)
dict_occ_kw_clean=util.get_occurance_dict(data_kw,['KEYWORD'])

removing rows from the df
occurance dictionary len: 77


In [23]:
set_p_ids_all=set(data_kw['PPID'])

In [24]:
def space_to_underscore(list_):
    result=[]
    for e in list_:
        result+=[e.replace(" ", "_")]
    return result

In [25]:
def get_tf_idf(corpus,df_min):
    vectorizer = TfidfVectorizer( token_pattern=r"(?u)\S\S+",min_df=df_min)
    X = vectorizer.fit_transform(corpus)
    feature_names = vectorizer.get_feature_names()
    return X, vectorizer

In [26]:
def get_doc_as_string_for_tf_idf(id_):
    row=data_kw.loc[data_kw['PPID']==id_]
    tien=list(literal_eval(row['TIEN'].iloc[0]))
    aben=list(literal_eval(row['ABEN'].iloc[0]))
    clm=list(literal_eval(row['CLMEN'].iloc[0]))

    detden=list(literal_eval(row['DETDEN'].iloc[0]))
    lst_all=space_to_underscore(tien+aben+clm)
    #lst_all=space_to_underscore(tien+aben+clm+detden)
    doc=" ".join([str(item) for item in lst_all])
    return doc

In [18]:
#prepare_corpus_for_tf_idf(set(test_set_dict.keys()))

In [27]:
def filter_np(lst_filtered_np, lst_to_be_filtered):
    return [e for e in lst_to_be_filtered if e in lst_filtered_np]

In [28]:
def prepare_corpus_for_tf_idf(set_ids):
    docs=dict()
    corpus=[]
    for i, id_ in enumerate(set_ids):
        doc=get_doc_as_string_for_tf_idf(id_)
        docs[id_]=doc
        corpus+=[doc]
    print('len corpus:', len(corpus))
    return corpus, docs
#print(feature_names)

In [29]:
#corpus_all=prepare_corpus_for_tf_idf(set_ids_all)
corpus_test, docs_ids_test =prepare_corpus_for_tf_idf(set(test_set_dict.keys()))
corpus_train, docs_ids_train = prepare_corpus_for_tf_idf(set_p_ids_all-set(test_set_dict.keys()))

len corpus: 100
len corpus: 5256


In [31]:
len(data_id_tit_abs_clm[data_id_tit_abs_clm['PPID']=='FR3050455A1']['ABEN'].iloc[0])

396

In [33]:
def precision_N_post(y_true, y_pred):
   # print(colored(sorted(y_true), 'red'))
    #print(colored(sorted(y_pred), 'green'))
    #print(colored((set(y_pred) & set(y_true)), 'blue'))
    #sys.exit()
    if len(set(y_pred))==0:
        return 0

    p=len(set(y_pred) & set(y_true)) * 1.0 / len(set(y_pred))
    if p==0:
        #print('y_pred',y_pred)
        #print('\n')
        #print('y_true',y_true)
        #print('\n\n\n')
        return 0
    #print(len(set(y_pred) & set(y_true)) * 1.0 / len(set(y_pred)))
    #print(y_pred)
    #print('\n')
    #print(y_true)
    #print('\n\n\n')
    return p

In [34]:
def recall_N_post(y_true, y_pred, N):
    #print(colored(y_true, 'red'))
    #print(colored(y_pred, 'green'))
    #print(colored((set(y_pred[:N]) & set(y_true)), 'blue'))
   
    return len(set(y_pred[:N]) & set(y_true)) * 1.0 / len(y_true)

In [35]:
def get_top_tf_idf_words(response, vectorizer, n=10):
    feature_array = np.array(vectorizer.get_feature_names())
    tfidf_sorting = np.argsort(response.toarray()).flatten()[::-1]
    top_n = feature_array[tfidf_sorting][:n]
    return top_n

In [37]:
X, vectorizer=get_tf_idf(corpus_train, df_min=50)
feature_names = vectorizer.get_feature_names()

In [38]:
len(feature_names)

1012

In [39]:
kws_considered=space_to_underscore(list(dict_occ_kw_clean.keys()))

In [40]:
len(kws_considered)

77

In [41]:
set(kws_considered)-set(feature_names)

set()

In [42]:
def f_score(precision, recall):
    return 2 * (precision * recall) / (precision + recall)

In [45]:
def calculate_precision_recall_N(test_set_dict, n):
   
    r = []
    p = []
    p_len = []
    hit = 0
    lst_preds=[]
    kws_considered=space_to_underscore(list(dict_occ_kw_clean.keys()))
    for i, (uid, gt) in enumerate(test_set_dict.items()):
        try:
            
            id_real=uid
            #print(id_real)
            #print(id_real, gt)
            set_gt_decoded=space_to_underscore(list(gt))
            
            df_temp=data.df_full_text.loc[data.df_full_text['PPID'] == id_real]
            
            df_nPha=data_id_tit_abs_clm.loc[data_id_tit_abs_clm['PPID'] == id_real]
            tien=df_nPha['TIEN'].iloc[0]
            aben=df_nPha['ABEN'].iloc[0]
            clmen=df_nPha['CLMEN'].iloc[0]
            
 
            
            #detden=df_nPha['DETDEN'].iloc[0]
            
            #set_full_kws=set(itertools.chain(literal_eval(tien), literal_eval(aben),literal_eval(clmen)))
            
            new_doc = get_doc_as_string_for_tf_idf(id_real)
            response = vectorizer.transform([new_doc])
            pred=get_top_tf_idf_words(response,vectorizer, n=len(new_doc.split())).tolist()
            
            #pred=get_top_tf_idf_words(response,vectorizer, n=77).tolist()
            
            #pred_post=[kw for kw in pred if kw in kws_considered][:n]
            #pred_post=[kw for kw in pred][:n]
            pred_post=[kw for kw in pred if kw not in lst_all][:n]
            
            #print(len(pred_post), pred_post)
            #sys.exit()
            #pred_post=[kw for kw in new_doc.split() if kw in kws_considered][:10]
            
            recall_score = recall_N_post(set_gt_decoded, pred_post, n)
            precision= precision_N_post( set_gt_decoded, pred_post)
            
            r.append(recall_score)
            
            #print(precision,recall_score)
            '''
            if id_real=='US20160185785A1':
                print(id_real)
                print(set_gt_decoded)
                print(len(set(pred_post)),pred_post)
                print('pre:',precision)
                print('recall:',recall_score)
                sys.exit()
            '''
            if len(set(pred_post))>0:
                p.append(precision)
                p_len.append(len(set(pred_post)))
                   
            hit += len(set(set_gt_decoded) & set(pred_post))
           
        except:
            print(i)
            sys.exit()
    print("recall", np.mean(r))
    print("precision", np.mean(p))
    print('f_score:', f_score(np.mean(p), np.mean(r)))

In [46]:
calculate_precision_recall_N(test_set_dict, n=10)

recall 0.2403497050707577
precision 0.3126666666666667
f_score: 0.2717798060216258


In [47]:
calculate_precision_recall_N(test_set_dict, n=15)

recall 0.3994686108582393
precision 0.35121212121212125
f_score: 0.37378931464043924


In [48]:
calculate_precision_recall_N(test_set_dict, n=20)

recall 0.5104358236849723
precision 0.34280035650623897
f_score: 0.41015040476495396
